In [2]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
sys.path.append('..')  
import model_helper 
from helper import *
from plotly_helper import *

In [3]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    abundance_df = pd.read_csv(participants.loc[patient_id]['abundances_tsv'], sep='\t')
    mcmc_df = pd.read_csv(participants.loc[patient_id]['cell_population_mcmc_trace'], sep='\t')
    tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Day'].tolist()]
    alc_all = [float(i) for i in wbc_df_patient['ALC'].tolist()]
    
    
    alc_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL_est'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Day'].to_list()]
    
    return wbc_df_patient, times_sample, alc_all,alc_count_sample,all_times
    





In [4]:
# Input patient information
# Input patient information
patient_id = 'RP-1895_FI-01'
wbc_file = '../combined_FI_alc_updated.csv'
treatment_file = '../GCLL_treatment.txt'
UMI_start = 7
UMI_end = 8
treatment_start = 0


In [5]:
# Manually verify the sample list order is correct

In [6]:

sample_list = ['RP-1895_8881-C1D1_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_8974-C2D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_9072-C3D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_9181-C4D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_9383-C6D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'SM-N57VF_RP-1895_273-SAL_v1_Exome_OnPrem_pair', 'SM-N57VG_RP-1895_273-SAL_v1_Exome_OnPrem_pair', 'SM-N57VC'] 

In [7]:
# Output file

In [8]:
outputfile_name = "FI_subplot_combined_01_wbc_updated.html"

Update patient specific information in the below cell 

In [9]:
# Get the treatment duration 

treatment_end = 2359

In [10]:
treatment_end

2359

In [11]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample,alc_all,alc_count_sample, all_times = filter_patient_data(wbc_df, patient_id)


# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, alc_count_sample, UMI_start, UMI_end, treatment_start, treatment_end,fludarabine_treatment_1=(57, 61),
               fludarabine_treatment_2=(84, 88)) 
    




In [12]:
len(all_times)

38

In [13]:
len(alc_all)


38

In [14]:
alc_count_sample

[247.06, 170.91, 60.3, 10.4, 5.03, 1.31712, 0.65688, 0.3267]

In [15]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [16]:
tree_df

,n_iter,likelihood,edges
0,249,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...","1-2,None-1"


In [17]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [18]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0
    ,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
 

)

In [19]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(alc_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [20]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [21]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(alc_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [22]:
times_aft_tx = [x for x in all_times if x > treatment_start]


# Set the extrapolate time after treatment
extrapolate_start_idx = 1


In [23]:
times_sample

[-1, 60, 97, 118, 181, 356, 706, 1441]

In [24]:
times_sample[extrapolate_start_idx:]

[60, 97, 118, 181, 356, 706, 1441]

In [25]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, alc_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end, CLL14_modeling=True)


In [26]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  alc_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end, CLL14_modeling=True)  

In [27]:


times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['Day'] > 0]['Day'].values  if int(i) > 0]

In [28]:
# Needs to manually check times_sliced_aft

In [29]:
times_sliced_aft

[27,
 55,
 60,
 69,
 76,
 97,
 100,
 102,
 118,
 125,
 153,
 181,
 272,
 356,
 440,
 531,
 615,
 706,
 790,
 874,
 972,
 1070,
 1161,
 1259,
 1350,
 1441,
 1541,
 1629,
 1729,
 1820,
 1910,
 2001,
 2092,
 2184,
 2359]

In [30]:
len(times_sliced_aft)

35

In [31]:
index_samples_model = slice(1,8)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [32]:
y

[None,
 None,
 array([4.99294031, 3.16019039]),
 None,
 None,
 array([3.94240288, 2.17130917]),
 None,
 None,
 array([2.15368114, 0.57973282]),
 None,
 None,
 array([1.38238399, 0.045563  ]),
 None,
 array([ 0.14121309, -1.79749551]),
 None,
 None,
 None,
 array([-0.51061842, -2.8666877 ]),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 array([-1.19493446, -3.72797189]),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [33]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Day > treatment_start]['CLL_est'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [34]:
wbc_model

[350.35,
 250.55,
 170.91,
 63.1,
 60.1,
 60.3,
 47.6,
 28.75,
 10.4,
 10.97,
 9.84,
 5.03,
 3.84,
 1.31712,
 1.22794,
 0.6174,
 0.84378,
 0.65688,
 0.63784,
 0.4998,
 0.47124,
 0.5474,
 0.40936,
 0.62356,
 0.50932,
 0.3267,
 0.37719,
 0.3564,
 0.3564,
 0.3267,
 0.3267,
 0.3564,
 0.3564,
 0.3861,
 0.4158]

In [35]:
len(wbc_model)

35

In [36]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [37]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model, CLL14_modeling=True )


DEBUG: plot_subclones_new_model called with clusters=[1, 2], times_sample[1]=60, treatment_end=2359


In [38]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end, CLL14_modeling=True)


In [39]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as FI_subplot_combined_01_wbc_updated.html
